<font face="Calibri" size="2"> <i>SBAE - Notebook Series - Part 2, version 0.1,  June 2022. Andreas Vollrath, UN-Food and Agricultural Organization, Rome</i>
</font>

![title](images/header.png)

# II - SBAE Time-Series Extraction & Change Detection
### Extract various time-series outputs for point data from Google Earth Engine
-------

This notebook takes you through the process of extracting outputs from various time-series change detection algorithms and structure them in a so-called data-frame (e.g. tabular structure).

### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

/home/vollrath/.local/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Area of Interest

# country/province case example
country = 'Buhweju'   
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', country)) # here any feature collection can be selected

# Asset case
#aoi = ee.FeatureCollection("my_aoi_feature_collection")

# Point Feature Collection from where to extract points
fc = ee.FeatureCollection("users/andreasvollrath/01_test_sbae_points").filterBounds(aoi)

# the column of a unique point identifier in your dataset
point_id_name = 'point_id'

grid_size = 0.25 # that's the size of the grid we are parallelizing on in degrees
workers = 10 # number of parallel EE requests

#### 2b - Check inputs visually

In [3]:
from sepal_ui.mapping import SepalMap

# create the grid
grid, grid_fc = h.processing_grid(aoi, grid_size, grid_size)

Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(grid_fc)
Map.addLayer(fc)
Map.addLayer(aoi)
Map

Styles()

SepalMap(center=[-0.3203418090924264, 30.347812392179087], controls=(ZoomControl(options=['position', 'zoom_in…

#### 2c - Time-series parameter settings

In [4]:
# start of calibration period (mainly for bfast)
start_calibration = "2015-01-01"  # YYYY-MM-DD format

# Actual period of interest, i.e. monitoring period
start_monitor = "2017-01-01"  # YYYY-MM-DD format
end_monitor =  "2018-01-01"  # YYYY-MM-DD format

# Directory where output and temp files will go
outdir = 'results'  # goes to module_results/sbae_point_analysis if left to None

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = True
global_products = True

# select the band to run the ts on
band='greenness' # other choices: ndfi, ndmi, mndwi, brightness, greenness, wetness

# select the resolution to which the satellite data will be resized.
scale=100  # in meters

### DO NOT CHANGE YET ###
satellite='Landsat'
max_cloud_cover = 75  # in percentage (0-100)

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [5]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

global_products = {
    'run': global_products,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': workers,
    'ts_params': {
        'start_calibration': start_calibration,
        'start_monitor': start_monitor,
        'end_monitor': end_monitor,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite,
        'scale': scale,
        'max_cc': max_cloud_cover
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'global_products': global_products
}

### 4 - Run the time-series data extraction

**Execute only**

In [6]:
h.get_change_data(aoi, fc, config_dict)

 Parallelizing time-series extraction on 10 threads for a total of 4 grid cells.
 Grid cell 0 does not contain any points. Going on with next grid cell.
 Processing gridcell 3
 Processing gridcell 2
 Processing gridcell 1


2022-07-13 11:56:48.214123: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-13 11:56:48.214172: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (6440da3b7416): /proc/driver/nvidia/version does not exist
2022-07-13 11:56:48.215384: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 Grid cell 2 with 1 points done in: 0:02:13.623958
 Grid cell 3 with 4 points done in: 0:03:44.106797
 Grid cell 1 with 22 points done in: 0:04:30.627884


/home/vollrath/.local/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Deleting temporary files
 Processing has been finished successfully. Check for final_results files in your output directory.
